In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.models import Inception3

from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Проверяем, доступны ли GPU
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(DEVICE)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(299), 
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225]),
    ])

trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                      download=True, transform=transform)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)

classes = tuple(str(i) for i in range(10))

In [ ]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0, 1000))
    im_val, label = testset[random_characters]
    imshow(im_val.data.cpu(), plt_ax=fig_x)

In [ ]:
model_extractor = Inception3(num_classes=10)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model_extractor.parameters())

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 3, 0.5)

model_extractor.to(DEVICE)

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    losses = []
  
    for idx, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        if idx % 50 == 0:
            losses.append(loss.item())
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc, losses

In [ ]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

In [ ]:
def train(model, criterion, opt, scheduler, epochs, batch_size):
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    all_losses = []

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            train_loss, train_acc, losses = fit_epoch(model, train_loader, criterion, opt)
            all_losses += losses
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))

            scheduler.step()
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history, all_losses

In [ ]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
history, total_loss = train(model=model_extractor, criterion=criterion, opt=optimizer, scheduler=scheduler, epochs=1 , batch_size=8)

In [ ]:
loss, acc, val_loss, val_acc = zip(*history)

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(18, 10))


gs = fig.add_gridspec(2,2)

ax_1 = fig.add_subplot(gs[0, 0])
ax_2 = fig.add_subplot(gs[0, 1])
ax_3 = fig.add_subplot(gs[1, :])

ax_1.set_title('Train Loass')
ax_1.set_xlabel('Epoch')
ax_1.set_ylabel('Loss')

ax_2.set_title('Val  Loss')
ax_2.set_xlabel('Epoch')
ax_2.set_ylabel('Loss')

ax_3.set_title('Every 50th loss')
ax_3.set_xlabel('Iteration / 50')
ax_3.set_ylabel('Loss')

sns.lineplot(ax=ax_1, data=loss);
sns.lineplot(ax=ax_2, data=val_loss);
sns.lineplot(ax=ax_3, data=total_loss);
